In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the image
image_path = 'Unmasked_Image.jpg'
image = mpimg.imread(image_path)

# Convert the image to grayscale
image_gray = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])

# Removing the upper and lower part of outer shell
image_gray[0:32,:] = np.zeros((32,image_gray.shape[1]))
image_gray[244: , :] = np.zeros((image_gray.shape[0]-244, image_gray.shape[1]))

# Display the grayscale image
plt.imshow(image_gray, cmap='gray')
plt.title('Grayscale Brain MRI')
plt.show()

# Convert the data to 1d array
brain_pixels = image_gray.reshape(-1)

# Histogram representing distribution of intensities 
plt.hist(brain_pixels, bins = 20, edgecolor = 'black')
plt.show()

#plotting the histogram for area containing the brain image
plt.hist(image_gray[42:230, 100:190],bins = 8 , edgecolor = 'black')
plt.show()

In [ ]:
# Determined the threshhold value by plotting the histogram for area containing the brain image 
threshold_value = 32

# Create a binary mask
brain_mask = image_gray > threshold_value

# Modify the binary mask ie. removing the shell row wise
for i in range(brain_mask.shape[0]):
    count =0
    for j in range(brain_mask.shape[1]-1):
        if brain_mask[i,j]==1 and brain_mask[i,j+1]==1:
            count +=1
        if brain_mask[i,j]== 1 and brain_mask[i,j+1]==0:
            count+=1
            if count<20:
                brain_mask[i,j+1-count: j+1] = np.zeros(count)
                count =0
            else: 
                count =0

#Adding part inside brain area which got masked by above code
brain_mask[42:230, 100:190]= np.where(brain_mask[42:230, 100:190] ==0, 1, brain_mask[42:230, 100:190] )
plt.imshow(brain_mask, cmap='gray')
plt.title('Brain Mask')
plt.show()


In [ ]:
final_image = image_gray * brain_mask
plt.imshow(final_image, cmap = 'gray')
plt.show()

In [ ]:
# K++ means clustering
K=3

brain_pixels = final_image.reshape(-1)

def compute_distance(point_1, point_2):
    return np.sqrt(np.sum((point_1 - point_2)**2))

# Initialize centroids 
def init_centr(brain_pixels, K):
    centroids = [brain_pixels[np.random.randint(brain_pixels.shape[0])]]
    for _ in range(K - 1):
        distances = np.array([min([compute_distance(pixel, centroid) for centroid in centroids]) for pixel in brain_pixels])
        next_centroid = brain_pixels[np.argmax(distances)]
        centroids.append(next_centroid)
    return np.array(centroids)

# Assigning pixels to the nearest centroid
def assign_clusters(brain_pixels, centroids, K):
    clusters = {i: [] for i in range(K)}
    for pixel in brain_pixels:
        distances = [compute_distance(pixel, centroid) for centroid in centroids]
        closest_centroid = np.argmin(distances)
        clusters[closest_centroid].append(pixel)
    return clusters

# Recompute centroids as the mean of assigned points
def recompute_centroids(clusters, K):
    new_centroids = [np.mean(clusters[i], axis=0) for i in range(K)]
    return np.array(new_centroids)

# Assigning each pixel to the nearest cluster's centroid
def assign_brain_pixels(brain_pixels, centroids, K):
    new_brain_pixels = np.zeros(brain_pixels.shape)
    for idx, pixel in enumerate(brain_pixels):
        distances = [compute_distance(pixel, centroid) for centroid in centroids]
        closest_centroid = np.argmin(distances)
        new_brain_pixels[idx] = centroids[closest_centroid]
    return new_brain_pixels

# Implementing K++
def kmeans_clust(brain_pixels, K, max_iterations=100):
    centroids = init_centr(brain_pixels, K)
    for _ in range(max_iterations):
        clusters = assign_clusters(brain_pixels, centroids, K)
        new_centroids = recompute_centroids(clusters, K)
        if np.all(centroids == new_centroids):
            break
        centroids = new_centroids
    return centroids, clusters


In [ ]:
final_centroids, final_clusters = kmeans_clust(brain_pixels, K)

# Assigning pixels to the closest centroid and reshape to the original image dimensions
clustered_brain_pixels = assign_brain_pixels(brain_pixels, final_centroids, K)
clustered_image = clustered_brain_pixels.reshape(image_gray.shape)

# Display the clustered image
plt.imshow(clustered_image.astype(np.uint8))
plt.show()